#  01. Assigment

This assigment, we will creat a script to acess edgar data and download a 1000 samples from 10-K

First we will import the dependencies and download a file from edgar with the links to the files tha we want to download.

This link https://www.sec.gov/edgar/searchedgar/accessing-edgar-data.htm has teh documentation and the file we want to download first

We will download the file 2017 fist quartil. The  file company.idx has the information. 

In [1]:
#Pandas dataframe is a very good pakage to work with files
import pandas as pd

In [2]:
#Here we create a variavle with the url with all url files
url_idx = 'https://www.sec.gov/Archives/edgar/full-index/2017/QTR1/company.idx'

In [3]:
#we use pandas direct to import the file from website into memory
#We use the function pd.read_fwf and skip the first 10 row that don't have any data 
#and without any header.

files_link = pd.read_fwf(url_idx, skiprows=10, header =None , \
                         colspecs=[(0,61),(61,73),(74,85), (86,96), (98,150)],
                        names= ['company_name','form_type','cik','date','file'])

In [4]:
#Let´s take a look how the data looks like
files_link.head()

,company_name,form_type,cik,date,file
0,"024 Pharma, Inc.",10-K,1307969,2017-03-24,edgar/data/1307969/0001683168-17-000653.txt
1,1 800 FLOWERS COM INC,10-Q,1084869,2017-02-10,edgar/data/1084869/0001437749-17-002102.txt
2,1 800 FLOWERS COM INC,3,1084869,2017-02-07,edgar/data/1084869/0001084869-17-000006.txt
3,1 800 FLOWERS COM INC,4,1084869,2017-02-03,edgar/data/1084869/0001084869-17-000001.txt
4,1 800 FLOWERS COM INC,4,1084869,2017-02-06,edgar/data/1084869/0001084869-17-000002.txt


How many files we have?

In [5]:
files_link['company_name'].count()

311453

WoW!!! More than 300k files. But we don´t need all.

In the collun 2 we can see the form type, we want only the 10-K's documents, so lets filter

In [6]:
documents = '10-K'

In [7]:
files_link_10k = files_link[files_link['form_type'] == documents]
files_link_10k.head()

,company_name,form_type,cik,date,file
0,"024 Pharma, Inc.",10-K,1307969,2017-03-24,edgar/data/1307969/0001683168-17-000653.txt
90,"1347 Property Insurance Holdings, Inc.",10-K,1591890,2017-03-16,edgar/data/1591890/0001387131-17-001422.txt
220,1ST CONSTITUTION BANCORP,10-K,1141807,2017-03-16,edgar/data/1141807/0001141807-17-000012.txt
238,1ST SOURCE CORP,10-K,34782,2017-02-17,edgar/data/34782/0000034782-17-000036.txt
265,1st FRANKLIN FINANCIAL CORP,10-K,38723,2017-03-31,edgar/data/38723/0001376474-17-000071.txt


How may files do we have now?

In [8]:
files_link_10k['company_name'].count()

5275

We need only 100 documents, we can download the first ones or we can sample randomly.
lets take a sample from this.

In [9]:
num_sample = 100

In [10]:
#we can set randon_state, so we can reproduce or sampling 
sample_files_link_10k = pd.DataFrame(files_link_10k.sample(n=num_sample,random_state=0),\
                                     copy=True)
sample_files_link_10k.reset_index(inplace=True)

In [11]:
sample_files_link_10k.head()

,index,company_name,form_type,cik,date,file
0,275433,TRAVELZOO INC,10-K,1133311,2017-03-15,edgar/data/1133311/0001133311-17-000010.txt
1,2116,ACCO BRANDS Corp,10-K,712034,2017-02-27,edgar/data/712034/0000712034-17-000012.txt
2,127781,GrowGeneration Corp.,10-K,1604868,2017-03-31,edgar/data/1604868/0001213900-17-003102.txt
3,16371,Advanced Biomedical Technologies Inc.,10-K,1385799,2017-02-14,edgar/data/1385799/0001387131-17-000831.txt
4,230264,"Primerica, Inc.",10-K,1475922,2017-02-27,edgar/data/1475922/0001564590-17-002594.txt


Now we can download the files.
First we need to put the beginig url and than download the files 

In [12]:
import urllib.request #to download the data from a website
import sys #to print 
import time #to get the time

count = 0
corpus = []
timeout = 15 #use this timeout to download the data. 
base_url = 'https://www.sec.gov/Archives/'

In [13]:
#start the clock. Lets se how long it takes to download our files
start = time.time()


for row in sample_files_link_10k['file']:
    count += 1
    baselink = base_url + row
    text = ''
    text = urllib.request.urlopen(baselink,timeout=5).read()
    corpus.append(text)
    end = time.time()
    
    
    # Printing out the the progress
    sys.stdout.write("\rProgress: {:2.1f}".format(100 * count/float(num_sample)) \
                     + "% ... Documents processed: " + str(count) \
                     + " time running: " + str(int((end-start)/60)) + " minutes.") 
    sys.stdout.flush()
    

Progress: 100.0% ... Documents processed: 100 time running: 3 minutes.

Now all files are saved as a list into memory. Lets transform to a Dataframe and put as a column in our previous dataframe.


In [14]:
sample_files_link_10k['corpus'] = pd.DataFrame(corpus)

In [15]:
sample_files_link_10k.head()

,index,company_name,form_type,cik,date,file,corpus
0,275433,TRAVELZOO INC,10-K,1133311,2017-03-15,edgar/data/1133311/0001133311-17-000010.txt,b'<SEC-DOCUMENT>0001133311-17-000010.txt : 201...
1,2116,ACCO BRANDS Corp,10-K,712034,2017-02-27,edgar/data/712034/0000712034-17-000012.txt,b'<SEC-DOCUMENT>0000712034-17-000012.txt : 201...
2,127781,GrowGeneration Corp.,10-K,1604868,2017-03-31,edgar/data/1604868/0001213900-17-003102.txt,b'<SEC-DOCUMENT>0001213900-17-003102.txt : 201...
3,16371,Advanced Biomedical Technologies Inc.,10-K,1385799,2017-02-14,edgar/data/1385799/0001387131-17-000831.txt,b'<SEC-DOCUMENT>0001387131-17-000831.txt : 201...
4,230264,"Primerica, Inc.",10-K,1475922,2017-02-27,edgar/data/1475922/0001564590-17-002594.txt,b'<SEC-DOCUMENT>0001564590-17-002594.txt : 201...


We have download all our files, but it is in memory. Let´s save into disk in one single file. So it will be very easy to import and work with this data after.

I will use .pickle that is a very good formart to save this kind of objects

In [16]:
#Saving the data for future use
sample_files_link_10k.to_pickle('./data/raw/10-K_Data.pik')

Now we can just load the file and wont need to import everything again.

If we download too much data, it is possible that we wont have enoght memomry in our computer, than we need to save every file and work in batchs.